In [2]:
!pip install mne

  Using cached tqdm-4.67.1-py3-none-any.whl.metadata (57 kB)
   ---------------------------------------- 0.0/7.4 MB ? eta -:--:--
   ---------------------------------------- 0.0/7.4 MB ? eta -:--:--
   -- ------------------------------------- 0.5/7.4 MB 2.6 MB/s eta 0:00:03
   ----- ---------------------------------- 1.0/7.4 MB 2.8 MB/s eta 0:00:03
   -------- ------------------------------- 1.6/7.4 MB 2.7 MB/s eta 0:00:03
   ------------ --------------------------- 2.4/7.4 MB 2.8 MB/s eta 0:00:02
   --------------- ------------------------ 2.9/7.4 MB 2.8 MB/s eta 0:00:02
   ------------------ --------------------- 3.4/7.4 MB 2.7 MB/s eta 0:00:02
   --------------------- ------------------ 3.9/7.4 MB 2.7 MB/s eta 0:00:02
   ------------------------- -------------- 4.7/7.4 MB 2.8 MB/s eta 0:00:01
   ---------------------------- ----------- 5.2/7.4 MB 2.8 MB/s eta 0:00:01
   ------------------------------- -------- 5.8/7.4 MB 2.8 MB/s eta 0:00:01
   --------------------------------- ----


[notice] A new release of pip is available: 25.0.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [5]:
!python.exe -m pip install --upgrade pip

   ---------------------------------------- 0.0/1.8 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.8 MB ? eta -:--:--
   ----- ---------------------------------- 0.3/1.8 MB ? eta -:--:--
   ----------------- ---------------------- 0.8/1.8 MB 2.2 MB/s eta 0:00:01
   ----------------------- ---------------- 1.0/1.8 MB 2.2 MB/s eta 0:00:01
   ----------------------------------- ---- 1.6/1.8 MB 2.2 MB/s eta 0:00:01
   ---------------------------------------- 1.8/1.8 MB 2.2 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 25.0.1
    Uninstalling pip-25.0.1:
      Successfully uninstalled pip-25.0.1


In [7]:
import numpy as np
import pandas as pd
import mne # MNE is a standard library for EEG data analysis
import os # to interact with the computer operating system (handling files, directories and paths

In [8]:
!pip install kagglehub

In [28]:
# Download the DEAP dataset from kaggle

import kagglehub
import pickle

# latest version

path = r"C:\Users\Sameera\.cache\kagglehub\datasets\manh123df\deap-dataset\versions\1"

print("Path to dataset files:", path)

Path to dataset files: C:\Users\Sameera\.cache\kagglehub\datasets\manh123df\deap-dataset\versions\1


In [29]:
# The 'path' variable from download (kaggle)
# navigate to the directory containing the subject files
DEAP_DATA_PATH = os.path.join(path, r"deap-dataset\data_preprocessed_python")

#List of subjects
subject_ids = [f's{i:02d}' for i in range(1,33)]

# verify the path
print(f"Looking for files in: {DEAP_DATA_PATH}")
print(f"First subject file expected: {os.path.join(DEAP_DATA_PATH, subject_ids[0] + '.dat')}")

Looking for files in: C:\Users\Sameera\.cache\kagglehub\datasets\manh123df\deap-dataset\versions\1\deap-dataset\data_preprocessed_python
First subject file expected: C:\Users\Sameera\.cache\kagglehub\datasets\manh123df\deap-dataset\versions\1\deap-dataset\data_preprocessed_python\s01.dat


In [30]:
# The 0-indexed positions of the 14 selected EEG channels in the DEAP file structure (0-31)

selected_indices = [0, 1, 2, 3, 6, 10, 12, 16, 17, 19, 20, 24, 28, 30]

def select_channels(eeg_data, indices):
    #eeg_data shape in (trials, total_channles, time_points) -> (40,40,8064) in raw DEAP
    #assume the EEG channels are the first 32
    eeg_32_channels = eeg_data[:, 0:32, :]

    #selects only the 14 desired EEG channels
    return eeg_32_channels[:, indices, :]

In [32]:
all_raw_data = []
all_labels = []

def load_subject_data(subject_id):
    file_path = os.path.join(DEAP_DATA_PATH, subject_id + '.dat')

    # The 'latin1' encoding is necessary for loading DEAP dataset files
    with open(file_path, 'rb') as f:
        subject_data = pickle.load(f, encoding='latin1')

    return subject_data

for subject_id in subject_ids:
    print(f"Processing {subject_id}...")

    subject_data = load_subject_data(subject_id)

    # Get the raw EEG data: shape (40 trials, 40 channels, 8064 time points)
    eeg_data = subject_data['data']

    # STEP 1: Channel Selection (14 Channels)
    data_14_channels = select_channels(eeg_data, selected_indices)

    # STEP 2: Label Discretization and One-Hot Encoding
    # Labels array has shape (40, 4) -> [Valence, Arousal, Dominance, Liking]
    labels_raw = subject_data['labels']
    valence = labels_raw[:, 0]
    arousal = labels_raw[:, 1]

    # Threshold for High/Low is >= 4.5 (paper 2)
    high_valence = (valence >= 4.5).astype(int)
    high_arousal = (arousal >= 4.5).astype(int)

    # Creates the 4-class one-hot labels
    labels_one_hot = np.zeroes((40, 4))

    # LALV : Low Arousal Low Valence -> [1, 0, 0, 0]
    labels_one_hot[(high_arousal == 0) & (high_valence == 0), 0] = 1
    # HALV : High Arousal Low Valence -> [0, 1, 0, 0]
    labels_one_hot[(high_arousal == 1) & (high_valence == 0), 1] = 1
    # LAHV : Low Arousal High Valence -> [0, 0, 1, 0]
    labels_one_hot[(high_arousal == 0) & (high_valence == 1), 2] = 1
    # HAHV : High Arousal High Valence -> [0, 0, 0, 1]

    all_raw_data.append(data_14_channels)
    all_labels.append(labels_one_hot)


# Final Conatenation (Accross all 32 subjects)
X_raw = np.concatenate(all_raw_data, axis=0)
Y_labels = np.concatenate(all_labels, axis=0)

print("\n--- Final RAW Dataset Summary ---")
print(f"Total RAW Data shape (Epochs, Channels, Time Points): {X_raw.shape}")
print(f"Total Labels shape (Epochs, 4 Classes): {Y_labels.shape}")


Processing s01...


AttributeError: module 'numpy' has no attribute 'zeroes'